<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Copy-tests-from-fastai2-notebook-12_optimizer.ipynb" data-toc-modified-id="Copy-tests-from-fastai2-notebook-12_optimizer.ipynb-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Copy tests from fastai2 notebook 12_optimizer.ipynb</a></span></li></ul></div>

In [1]:
from fastai2.vision.all import *

In [2]:
torch.optim.SGD?

### Copy tests from fastai2 notebook 12_optimizer.ipynb

In [3]:
tst_sgd = OptimWrapper(torch.optim.SGD([{'params': [tensor([1,2,3])], 'lr': 1e-3}, 
                                        {'params': [tensor([4,5,6])], 'lr': 1e-2}], momentum=0.9, weight_decay=1e-2))

In [4]:
tst_sgd.hypers

[{'lr': 0.001, 'mom': 0.9, 'dampening': 0, 'wd': 0.01, 'nesterov': False},
 {'lr': 0.01, 'mom': 0.9, 'dampening': 0, 'wd': 0.01, 'nesterov': False}]

In [20]:
L(tst_sgd.hypers)

(#2) [{'lr': 0.001, 'mom': 0.9, 'dampening': 0, 'wd': 0.01, 'nesterov': False},{'lr': 0.01, 'mom': 0.9, 'dampening': 0, 'wd': 0.01, 'nesterov': False}]

In [5]:
class OptFuncWrapper:
    def __init__(self, f):
        self.f = f
    def __call__(self, *args, **kwargs):
        opt = self.f(*args, **kwargs)
        optim_wrapper = OptimWrapper(opt)
        return optim_wrapper
     

In [6]:
SGD

<function fastai2.optimizer.SGD(params, lr, mom=0.0, wd=0.0, decouple_wd=True)>

In [7]:
torch.optim.SGD

torch.optim.sgd.SGD

In [8]:
sgd = SGD([[tensor([1,2,3])], [tensor([4,5,6])]], lr=[1e-3, 1e-2], mom=0.9, wd=1e-2)
#Access to param_groups

In [9]:
param1 =  [tensor([1,2,3]),tensor([-1,-2,-3])]
param2 =  [tensor([4,5,6]),tensor([-4,-5,-6])]

In [10]:
params = [param1,param2]

In [11]:
my_sgd = SGD(params=params,lr=[3e-3,2e-2], mom=0.7,wd=5e-3)

In [12]:
my_sgd.hypers

(#2) [{'wd': 0.005, 'mom': 0.7, 'lr': 0.003},{'wd': 0.005, 'mom': 0.7, 'lr': 0.02}]

In [14]:
torch_sgd = torch.optim.SGD(params=params[0],lr=3e-3,momentum=0.7,weight_decay=5e-3)

In [15]:
my_torch_sgd = OptimWrapper(torch_sgd)

In [16]:
my_torch_sgd.hypers

[{'lr': 0.003, 'mom': 0.7, 'dampening': 0, 'wd': 0.005, 'nesterov': False}]

In [17]:
torch_sgd2 = torch.optim.SGD([{'params':params[0],'lr':3e-3},
                              {'params':params[1],'lr':2e-2}], momentum=0.7,weight_decay=5e-3)

In [18]:
my_torch_sgd2 = OptimWrapper(torch_sgd2)

In [19]:
my_torch_sgd2.hypers

[{'lr': 0.003, 'mom': 0.7, 'dampening': 0, 'wd': 0.005, 'nesterov': False},
 {'lr': 0.02, 'mom': 0.7, 'dampening': 0, 'wd': 0.005, 'nesterov': False}]

In [21]:
!pip freeze | grep fastai_xla_extensions

In [22]:
def my_dumb_step(opt):
    print('this is my dumb step')
    opt.step()
    
class MyDumbOptimProxy:
    def __init__(self,opt):
        self.opt = opt
        
    def my_own_step(self):
        my_dumb_step(self.opt)
        
    def __getattr__(self,name):
        if name == 'step':
                return getattr(self,'my_own_step')
        return getattr(self.opt,name)

In [23]:
sgd.step()

In [24]:
mysgd_proxy = MyDumbOptimProxy(sgd)

In [25]:
mysgd_proxy.hypers

(#2) [{'wd': 0.01, 'mom': 0.9, 'lr': 0.001},{'wd': 0.01, 'mom': 0.9, 'lr': 0.01}]

In [26]:
mysgd_proxy.step()

this is my dumb step


In [ ]:
test_eq(tst_sgd.param_lists, sgd.param_lists)
#Set param_groups

In [ ]:
tst_sgd.param_lists = [[tensor([4,5,6])], [tensor([1,2,3])]]

In [ ]:
test_eq(tst_sgd.opt.param_groups[0]['params'], [tensor(4,5,6)])

In [ ]:
test_eq(tst_sgd.opt.param_groups[1]['params'], [tensor(1,2,3)])

In [ ]:
#Access to hypers
test_eq(tst_sgd.hypers, [{**sgd.hypers[i], 'dampening': 0., 'nesterov': False} for i in range(2)])

In [ ]:
#Set hypers
tst_sgd.set_hyper('mom', 0.95)

In [ ]:
test_eq([pg['momentum'] for pg in tst_sgd.opt.param_groups], [0.95,0.95])

In [ ]:
tst_sgd.set_hyper('lr', [1e-4,1e-3])

In [ ]:
test_eq([pg['lr'] for pg in tst_sgd.opt.param_groups], [1e-4,1e-3])

### New heading